# Comparación de grupos
*"La mitad de mi gasto en publicidad no sirve para nada. El problema es que no sé qué mitad" atribuida a John Wanamaker(1838-1922)*

## Pruebas A/B
En el área de negocios, especialmente en innovación digital, es común realizar pruebas A/B para evaluar las respuestas de los usuarios a diferentes alternativas en el diseño de un sitio o una interfaz.  
Entre sus características básicas está que los usuarios son asignados al azar a un grupo de control (A) o a un grupo experimental (B). A cada grupo se les muestra una alternativa diferente y se recopilan diversas métricas. Finalmente, analizando los resultados, se concluye qué alternativa tuvo mejor desempeño.  
En la siguiente liga se muestran varios ejemplos de aplicaciones prácticas:
https://goodui.org/leaks/. Este tipo de experimentos ejemplifica la aplicación de técnicas que veremos en esta sesión relacionadas con la comparación de dos o más grupos

El archivo "cookie_cats.csv" contiene información de 90,189 jugadores. Cuando un jugador instala el juego empieza en el nivel 30, sin embargo, en la empresa que diseñó el juego han propuesto que el jugador empiece en el nivel 40. Para evaluar ambas opciones se diseñó una prueba A/B en la cual los jugadores fueron asignados aleatoriamente para comenzar en el nivel 30 (control) o en el nivel 40 (tratamiento).

<div style="text-align: center;">
<img src="images/cookie_cats.jpg" alt="Cookie cats" width="300" height="200">
</div>

Las variables en el dataset son:

- userid: identificador del jugador.
- version: gate30, empieza en el nivel 30; gate40: empieza en el nivel 40.
- sum_gamerounds: número de rondas jugadas durante los primeros 14 días.
- retention_1: el jugador jugó un día después de instalar el juego.
- retention_7_ el jugador jugó después de 7 días de haber instalado el juego.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/cookie_cats.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [3]:
df.describe()

,userid,sum_gamerounds
count,9.018900e+04,90189.000000
mean,4.998412e+06,51.872457
std,2.883286e+06,195.050858
min,1.160000e+02,0.000000
25%,2.512230e+06,5.000000
50%,4.995815e+06,16.000000
75%,7.496452e+06,51.000000
max,9.999861e+06,49854.000000


In [4]:
# Eliminación de datos atípicos
df = df[df['sum_gamerounds']<1000]

## Prueba z para dos proporciones
Realizaremos una prueba z para comparar dos proporciones. Las pruebas de hipótesis pueden ser bilaterales (de dos colas) o unilaterales (de una cola). Utilizaremos una prueba bilateral, por lo tanto, las hipótesis se plantean como:
- H<sub>0</sub>: p=p<sub>0</sub> (Hipótesis nula)
- H<sub>1</sub>: p ≠ p<sub>0</sub> (Hipótesis alternativa)

Para calcular las proporciones conviene obtener una tabla con el conteo de cada caso. Realicemos una tabla de contigencia o tabla cruzada. 

In [5]:
# Crear una tabla de contingencia
tabla_contingencia = pd.crosstab(df['version'], df['retention_1'])
tabla_contingencia

retention_1,False,True
version,,
gate_30,24663,19984
gate_40,25368,20056


Utilizaremos estos conteos como datos para realizar la prueba z

In [6]:
from statsmodels.stats.proportion import proportions_ztest

# Contar el número de éxitos y el tamaño de cada grupo
conteos = df.groupby('version')['retention_1'].sum().values #suma de unos o conteo de éxitos
sumas = df.groupby('version')['retention_1'].count().values # conteo total de casos

# Realizar la prueba z para comparar proporciones
stat, p_value = proportions_ztest(conteos, sumas)

# Mostrar los resultados
print(f'Estadístico z: {stat}')
print(f'Valor p: {p_value}')

Estadístico z: 1.8333709789201535
Valor p: 0.06674742038798634


En las pruebas estadísticas aplicaremos el criterio del p-valor, esto es, rechazar la hipótesis nula si el p-valor es menor al nivel de significancia (típicamente se utiliza 0.05)

In [7]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de que hay diferencias significativas en las proporciones.")
else:
    print("No se rechaza la hipótesis nula: no hay suficiente evidencia para afirmar que hay diferencias significativas en las proporciones.")

No se rechaza la hipótesis nula: no hay suficiente evidencia para afirmar que hay diferencias significativas en las proporciones.


## Prueba t para dos muestras independientes

Los promedios también son una métrica comúnmente utilizada en las pruebas A/B. Comparemos las rondas de juego en promedio por cada grupo en el experimento.

In [8]:
# Estadística descriptiva
df.groupby('version').agg({'sum_gamerounds':'mean'})

,sum_gamerounds
version,
gate_30,49.825789
gate_40,49.487760


Consideraciones previas para realizar una prueba t:  
- la variable debe tener escala de medición de intervalo o de razón
- la variable debe tener una distribución normal o la muestra debe ser grande (mayor a 30)
- las observaciones deben ser independientes.

Planteamiento de hipótesis:  
$H_0: \mu_1 = \mu_2$  
$H_1: \mu_1 \neq \mu_2$

In [9]:
from scipy.stats import ttest_ind

# Filtramos los datos para obtener los grupos
control = df[df['version'] == 'gate_30']['sum_gamerounds']
experimental = df[df['version'] == 'gate_40']['sum_gamerounds']

# Realizar la prueba t de dos muestras independientes
t_statistic, p_value = ttest_ind(control, experimental)

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

Estadístico t: 0.5584128848228552
Valor p: 0.5765638644688713


El p-valor reportado corresponde a una prueba bilateral, es decir, cuando no se anticipa si la diferencia será positiva o negativa

In [10]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las medias.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.")

No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en las medias.


Si la muestra es pequeña (p<30) se requiere evaluar la normalidad en la distribución de los datos. Existen varias pruebas para evaluar la normalidad de los datos. La prueba Shapiro-Wilk es una de las más utilizadas.  
- Hipótesis nula (H0): Los datos provienen de una distribución normal  
- Hipótesis alternativa (H1): Los datos no provienen de una distribución normal.  

## Prueba de Levene
La prueba de Levene de homogeneidad de varianzas evalúa si la varianza es la misma para los diferentes grupos. Se debe revisar el p-valor (significancia), si este es mayor que 0.05 entonces no rechaza la hipótesis nula de igualdad de varianzas (es decir, no se está violando el supuesto de homogeneidad de varianzas)

In [11]:
from scipy.stats import levene

# Filtramos los datos para obtener los grupos
control = df[df['version'] == 'gate_30']['sum_gamerounds']
experimental = df[df['version'] == 'gate_40']['sum_gamerounds']

# Realizar la prueba 
stat, p_value = levene(control, experimental)
print("Estadístico de prueba:", stat)
print("Valor p:", p_value)

Estadístico de prueba: 0.03880220384954784
Valor p: 0.8438414811744965


In [12]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en las varianzas.")
else:
    print("No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.")

No se puede rechazar la hipótesis nula: no se rechaza el supuesto de igualdad de varianzas.


## Prueba t de Welch 
Si hay evidencia estadística de que las varianzas son diferentes, se recomienda hacer una prueba t de Welch (para varianzas diferentes)

In [13]:
from scipy.stats import ttest_ind

# Filtramos los datos para obtener los grupos
control = df[df['version'] == 'gate_30']['sum_gamerounds']
experimental = df[df['version'] == 'gate_40']['sum_gamerounds']

# Realizar la prueba t de dos muestras independientes
t_statistic, p_value = ttest_ind(control, experimental, equal_var = False)

# Imprimir los resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

Estadístico t: 0.558396552424921
Valor p: 0.5765750151466942


## Prueba Mann-Whitney
Una alternativa no paramétrica a la prueba t es la prueba de Mann-Whitney. Esto es recomendable cuando no se cumple el supuesto de normalidad de los datos.

In [14]:
from scipy.stats import mannwhitneyu

# Filtramos los datos para obtener los grupos
control = df[df['version'] == 'gate_30']['sum_gamerounds']
experimental = df[df['version'] == 'gate_40']['sum_gamerounds']

# Realizar la prueba de Mann-Whitney U
statistic, p_value = mannwhitneyu(control, experimental)

# Imprimir los resultados
print("Estadístico de prueba U:", statistic)
print("Valor p:", p_value)

Estadístico de prueba U: 1021921905.5
Valor p: 0.042826414935189544


In [ ]:
# Interpretar los resultados
alpha = 0.05
if p_value < alpha:
    print("Se rechaza la hipótesis nula: hay evidencia de diferencias significativas en la distribución.")
else:
    print("No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar diferencias significativas en la distribución")

## Ejercicio
Utiliza la base de datos "enigh_2020" para evaluar si el ingreso corriente por hogar en promedio es diferente entre los hogares donde el jefe de familia es hombre en comparación con los hogares donde el jefe de familia es mujer.

- Cargar los datos y librerías, explorar los datos
- Evaluar supuestos: igualdad de varianzas y normalidad (no se requiere en este caso)
- Realizar la prueba correspondiente

## Referencias
https://roirevolution.com/blog/why-ab-testing-could-save-your-marketing-strategy/